##LSH


In [ ]:
!pip install lshashing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from lshashing import LSHRandom
import numpy as np

# Loading the data
lsh_data = np.random.randint(size = (10, 10), low = 0, high = 50)
lsh_point = np.random.randint(size = (1,10), low = 0, high = 50)

# Applying the LSH appropriate nearest neighbor search
lshashing = LSHRandom(lsh_data, hash_len = 3, num_tables = 2)

# Displaying data
print(lshashing.tables[1].hash_table)
print(lshashing.knn_search(lsh_data, lsh_point[0], k = 3, buckets = 2, radius = 2))

{25: [0, 1, 2, 9], 49: [3, 4, 6, 7, 8], 16: [5]}
[[50.43808085  5.        ]
 [49.45705208  8.        ]
 [49.45705208  9.        ]]


In [ ]:
# Loading the data
lsh_data = np.random.randint(size = (10, 10), low = 0, high = 50)
lsh_point = np.random.randint(size = (1, 10), low = 0, high = 50)

# Applying the LSH appropriate nearest neighbor search
lshg_random_parallel = LSHRandom(lsh_data, 4, parallel = True)
lshg_random_parallel.knn_search(lsh_data, lsh_point[0], 5, 2, parallel = True)

[Neighbor(index=1, distance=35.369478367654786, value=[[26  9 16]...]),
 Neighbor(index=5, distance=45.552167895721496, value=[[31 33 16]...]),
 Neighbor(index=2, distance=46.78675026115834, value=[[19 39 10]...]),
 Neighbor(index=0, distance=51.39066063011839, value=[[26 45 25]...]),
 Neighbor(index=6, distance=53.54437412091022, value=[[8 2 0]...])]

##Exhaustive Search

In [ ]:
# Compute the squared Euclidean distance
def SEDist(X, Y):
    return sum((i-j)**2 for i, j in zip(X, Y))

SEDist( (9, 7), (3, 5) )

40

In [ ]:
# Calculating the nearest neighbor with the reference points
def exhaustive_search(*, es_points, exhaustive_points):
    return {
        query_p: min(
            exhaustive_points,
            key=lambda X: SEDist(X, query_p),
        )
        for query_p in es_points
    }

# Defining the data
exhaustive_points = [ (2, 3), (1, 5), (3, 4), (5, 10) ]
es_points = [
    (11, 3), (2, 7), (8, 5), (1, 10), (17, 15), (7, 13), (7, 9)
]

exhaustive_search(
    exhaustive_points = exhaustive_points,
    es_points = es_points,
)

{(11, 3): (3, 4),
 (2, 7): (1, 5),
 (8, 5): (3, 4),
 (1, 10): (5, 10),
 (17, 15): (5, 10),
 (7, 13): (5, 10),
 (7, 9): (5, 10)}

## Product Quantization

In [ ]:
!pip install nanopq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import libraries
import nanopq
import numpy as np

# Loading the data
val1 = np.random.random((300, 10)).astype(np.float32)
val2 = np.random.random((200, 10)).astype(np.float32)
pq_query_data = np.random.random((10, )).astype(np.float32)

In [ ]:
pq_data = nanopq.PQ(M=2, Ks=10, verbose=True)

M: 2, Ks: 10, code_dtype: <class 'numpy.uint8'>


In [ ]:
pq_data = nanopq.PQ(M=2, Ks=10).fit(vecs=val2, iter=20, seed=111)

M: 2, Ks: 10, code_dtype: <class 'numpy.uint8'>
iter: 20, seed: 111
Training the subspace: 0 / 2
Training the subspace: 1 / 2


In [ ]:
# Vectors encoded to PQ-codes.
value1_code = pq_data.encode(vecs=val1)

Encoding the subspace: 0 / 2
Encoding the subspace: 1 / 2


In [ ]:
pq_datatable = pq_data.dtable(query=pq_query_data) 
new_pq = pq_datatable.adist(codes=value1_code) 
new_pq = pq_data.dtable(query=pq_query_data).adist(codes=value1_code) 
min_value = np.argmin(new_pq)

In [ ]:
# The results by PQ
print(new_pq[:30])

# The results by the exact scan
new_pq_exact = np.linalg.norm(val1 - pq_query_data, axis=1) ** 2
print(new_pq_exact[:30])

[0.52229726 1.3840045  1.2289101  1.4527218  1.1012701  1.5803618
 0.83673996 1.1996658  1.3268378  0.91965926 1.0243659  1.4906436
 0.97956365 1.2856088  0.62769973 0.60635126 1.4284325  1.5960867
 0.5743052  1.7023486  1.6749823  0.5294471  1.0117388  1.3268378
 0.79214025 0.7864803  0.58240426 1.453263   1.5010902  0.58284163]
[1.1200771  2.0342693  2.3752615  1.4778057  1.4892058  1.3779052
 1.6327633  2.1839707  1.394433   0.9513964  1.4154241  1.5987351
 1.2298341  2.2654498  0.57176816 0.61424017 2.2193167  1.5456489
 0.6384673  1.7476935  2.6706524  1.2252753  1.6749492  1.6958979
 0.94229716 0.70993567 1.2702565  1.5556422  1.3933926  0.34002504]


In [ ]:
# Vectors approximately reconstructed by fetching codewords
value1_reconstructed = pq_data.decode(codes=value1_code)
print(val1[:3])
print(value1_reconstructed[:3])

[[0.4754867  0.95685625 0.08172247 0.18975958 0.6016347  0.30579254
  0.6178802  0.31128013 0.13745235 0.32055143]
 [0.7332759  0.24362962 0.9150325  0.6556154  0.06830217 0.3880876
  0.09113412 0.98937243 0.2645031  0.29095256]
 [0.0066501  0.8551826  0.68901825 0.10506567 0.6549309  0.10277326
  0.08737739 0.2743617  0.9602649  0.0548634 ]]
[[0.72307533 0.8006407  0.18633671 0.3181567  0.735876   0.25026757
  0.7935566  0.27955455 0.4047771  0.3746541 ]
 [0.7304915  0.34204268 0.7969214  0.6447078  0.126326   0.6889444
  0.3129007  0.54384756 0.20302553 0.47358915]
 [0.18935557 0.740808   0.32463908 0.25170764 0.5721659  0.30446324
  0.19499442 0.36450067 0.6951364  0.23501287]]


## Trees and Graphs

In [ ]:
# Importing libraries 
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Loading the data
data = np.array([[-11, -5], [-8, -10], [-7, -1], [5, 3], [4, 1], [9, 8]])

# Finding nearest neighbors with ball tree approach
tg_neighbors = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(data)
tg_distances, tg_indices = tg_neighbors.kneighbors(data)
tg_indices
tg_distances

array([[0.        , 5.65685425],
       [0.        , 5.83095189],
       [0.        , 5.65685425],
       [0.        , 2.23606798],
       [0.        , 2.23606798],
       [0.        , 6.40312424]])

In [ ]:
tg_neighbors.kneighbors_graph(data).toarray()

array([[1., 0., 1., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 0., 1.]])

## HNSW

In [ ]:
!pip install n2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 26.5 MB/s 
  Created wheel for n2: filename=n2-0.1.7-cp37-cp37m-linux_x86_64.whl size=2299813 sha256=4b9718721050006d7a22de42019445736e439d690d1d83da030dd1b02a3e2cd1
  Stored in directory: /root/.cache/pip/wheels/36/da/12/b157ca1c9dcdd5fd3fa5e15b7823f805396fb6e6b30427465a
Successfully built n2


In [ ]:
# Importing Libraries 
import numpy as np
from n2 import HnswIndex

# Loading data
Values, reference = 20240, 20
hnsw_data = np.arange(Values * reference).reshape(Values, reference)

# Applying HNSW approach
hnsw_index_data = HnswIndex(reference)
for data in hnsw_data:
    hnsw_index_data.add_data(data)
hnsw_index_data.build(m=5, n_threads=3)
print(hnsw_index_data.search_by_id(20, 36))

[20, 21, 19, 22, 18, 23, 24, 17, 25, 26, 16, 27, 28, 29, 15, 30, 31, 32, 33, 34, 14, 35, 36, 37, 38, 39, 40, 41, 42, 13, 43, 44, 45, 46, 47, 48]
